In [4]:
pip install --upgrade bt

Note: you may need to restart the kernel to use updated packages.


In [6]:
import bt

In [7]:
data = bt.get('SPY, IEV, EWJ, EEM, TLT, IEF, IYR',
              start='2010-01-01', end='2022-12-31')
data

,spy,iev,ewj,eem,tlt,ief,iyr
Date,,,,,,,
2010-01-04,86.315987,26.106081,31.653191,31.608870,59.726517,65.161797,27.733002
2010-01-05,86.544472,26.132189,31.843309,31.838289,60.112175,65.447914,27.799603
2010-01-06,86.605370,26.164827,31.970045,31.904903,59.307480,65.183823,27.787498
2010-01-07,86.970978,26.079971,31.684879,31.719879,59.407280,65.183823,28.035686
2010-01-08,87.260406,26.288822,32.065105,31.971500,59.380688,65.264542,27.848026
...,...,...,...,...,...,...,...
2022-12-23,373.653900,43.252716,53.249134,36.557606,95.533218,91.256584,80.635956
2022-12-27,372.180389,43.338272,53.006077,37.099197,93.644249,90.492561,80.578728
2022-12-28,367.554962,42.891483,52.374126,36.557606,93.092514,90.275620,79.214600


In [9]:
strategy = bt.Strategy('Asset_EW',
                       [bt.algos.SelectAll(),
                        bt.algos.WeightEqually(),
                        bt.algos.RunMonthly(),
                        bt.algos.Rebalance()
                        ])

AttributeError: module 'bt.algos' has no attribute 'WeightEqually'